# <span style="color: blue">Laboratorio 9: Asociación</span> <a class="tocSkip"> 

### <span style="color: green">Prof. Andrés Arguedas - XS-0300 - II 2020</span> <a class="tocSkip"> 

*Este laboratorio es parte del curso XS-0300 Estadística para Biología I de la Escuela de Estadística de la Universidad de Costa Rica, y se presenta con una licencia [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/).*

**ATENCIÓN:** Antes de entregar este laboratorio, asegúrese que todo corre como se espera. Para hacer esto, primero **reinice el kernel**, esto lo puede hacer seleccionado **Kernel** en el menú y luego **Restart**. Habiendo reiniciado el kernel, **corra todas las celdas**, esto lo puede hacer seleccionando **Cell** en el menú y luego **Run All**.

Por último, recuerde rellenar cualquier lugar donde diga `ESCRIBA SU CÓDIGO AQUÍ` o "ESCRIBA SU RESPUESTA AQUI", así como escribir su nombre y carné a continuación:

In [ ]:
NOMBRE = ""
CARNE = ""

---

## Preparación para este laboratorio
    
El presente laboratorio tiene como objetivos de aprendizaje:
    
1. Repasar el uso de gráficos de dispersión para determinar la relación entre dos variables
2. Aprender sobre el uso de la función `cor()` para calcular correlaciones, además de como interpretar y graficar estas correlaciones lineales
3. Conocer la forma de realizar pruebas de hipótesis e intervalos de confianza sobre la correlacion

En este laboratorio utilizaremos el conjunto de datos `doctors-and-life-expectancy`, junto con los paquetes `ggplot2`, `readr`, `dplyr`, `GGally` y `corrplot`. Ya todos estos paquetes están descargados y disponibles en el entorno virtual, al igual que el conjunto de datos, por lo que no es necesario que descarguen o consigan ningún paquete o conjunto de datos adicional, todos ya están disponibles.

---

## Conjunto de datos

Para este laboratorio utilizaremos un conjunto de datos que contiene, para una serie de países, la esperanza de vida, junto con otras dos variables: la raíz cuadrada de la cantidad de televisores per cápita y la raíz cuadrada de la cantidad de doctors per cápita. La razón para utilizar la raíz cuadrada es para poder tener una relación lineal y controlar por valores extremos, ya sea muy grandes o muy pequeños. En este caso, los datos se encuentran en un archivo `.txt` y están delimitados por una tabulación por lo que, en este caso, podemos usar la función `read_delim()` del paquete `readr`, pero tenemos que agregar algunos argumentos adicionales. Para efectos de este laboratorio, solamente es necesario correr la siguiente celda de código para cargar los datos y hacer un resumen de ellos:

In [ ]:
library(readr)

datos <- read_delim("data/doctors-and-life-expectancy.txt", "\t", escape_double = FALSE, trim_ws = TRUE)

summary(datos)

Como podemos ver, el conjunto de datos contiene cuatro variables:

- `country`: el nombre del país
- `life.exp`: la esperanza de vida en el país
- `sqrtTV.person`: la raíz cuadrada de la cantidad de televisores per cápita
- `sqrtDoctors.person`: la raíz cuadrada de la cantidad de doctores per cápita

Cabe mencionar que hay dos valores faltantes en las variables `life.exp` y `sqrtTV.person`. Adicionalmente, procederemos a eliminar la variable `country`, dado que solamente sirve como identificación, por lo que, al no ser numérica, nos puede presentar problemas más adelante. Procedamos a eliminar dicha variable y verificar que los cambios se realizaron como esperábamos:

In [ ]:
library(dplyr)

datos <- datos %>% select(-country)

summary(datos)

Dado que ya hemos cargado los datos y que eliminamos las variables que ocupabamos eliminar, podemos proceder a analizar los datos de forma descriptiva con un poco más de profundidad.

---

## Gráficos de dispersión y correlación

Los gráficos de dispersión son formas de graficar dos variables numéricas, una contra otra. Este tipo de gráficos nos permite determinar si existe una relación entre ambas variables y, en caso de existir, que forma tiene dicha relación. Aunque podemos hacer gráficos de dispersión con todas las variables manualmente, uno a uno, esto se puede volver bastante cansado. Por lo anterior, hay algunas funciones en R que permiten hacer gráficos de dispersión entre **todas** las variables de un conjunto de datos. En R base, se puede usar la función `pairs()`, mientras que, el equivalente en ggplot, sería la función `ggpairs()` del paquete `GGally`. Por lo tanto, procedamos a crear los gráficos de dispersión entre todas las variables de nuestro conjunto de datos:

In [ ]:
library(GGally)

ggpairs(datos)

La ventaja de utilizar la función anterior es que no solamente nos devuelve todos los gráficos de dispersión entre las variables del conjunto de datos, sino que también nos da una especie de histograma en la diagonal, junto con la correlación en el triángulo superior. Viendo los gráficos de dispersión, solamente nos interesa, por ahora, la relación de las variables con la esperanza de vida. A simple vista, parece que hay una relación positiva entre la esperanza de vida y la raíz cuadrada de los televisores per cápitda y la raíz cuadrada de los doctores per cápita. Es importante recordar que estas relaciones no tienen que ser perfectas, sino que, en promedio, podemos decir que, conforme aumenta la raíz cuadrada de la cantidad de televisores, o doctores, per cápita, también se da un aumento en la esperanza de vida. Por lo tanto, aunque esta relación no se cumple para todos los datos en el gráfico, si se nota una *tendencia* positiva. Analizando las correlaciones en el triángulo superior del gráfico anterior, podemos notar que devuelve un valor de la correlación, denominado `Corr:`, junto con una serie de asteriscos. En este caso, estos asteriscos representan si la correlación es o no significativa, aunque hablaremos más sobre esto posteriormente en este laboratorio. Pero entonces, ¿cómo hacemos para obtener el cálculo de estas correlaciones en R? Para esto, podemos hacer uso de la función `cor()` la cual, si le damos un conjunto de datos **con todas las variables numéricas**, nos devuelve la correlación entre todas las variables:

In [ ]:
cor(datos)

Notemos que en la matriz anterior, en la diagonal todos los valores son de 1. ¿Por qué sucede esto? Recordemos que una correlación de 1 indica una correlación lineal positiva perfecta, es decir, por cada aumento en X, se produce un aumento proporcional en Y. En el caso que X y Y sean la misma variable, tiene sentido pensar que, por cada aumento en X se va a producir un aumento igual en X, por lo que la correlación tiene que ser 1. Esto quiere decir que, cuando obtengamos un resultado de este estilo para las correlaciones, no tiene mucho sentido interpretar la correlación. Por otro lado, si analizamos la esquina superior derecha y la inferior izquierda, podemos notar que el valor es igual en ambas posiciones. ¿Por qué sucede esto? Recordemos que la correlación mide la relación lineal entre las variables X y Y, pero no nos importa cual variable es la dependiente o independiente, sino que podemos cambiarlas sin ninguna repercusión. Esto significa que la correlación entre X y Y es la misma que la correlación entre Y y X. Por último, ¿por qué sale `NA` en la matriz de correlaciones? Si recordamos, habían dos valores faltantes en la variable `sqrtTV.person` por lo que, al igual que pasaba con el promedio y la desviación estándar, el resultado nos da `NA`. Por lo anterior, deberíamos eliminar estos valores faltantes o, de forma más sencilla, especificarle a R que solamente tome las observaciones completas, mediante el argumento `use = "complete.obs"`, como se muesta a continuación:

In [ ]:
mat.cor <- cor(datos, use = "complete.obs")

mat.cor

Parece que, ahora sí, tenemos la correlación entre todas las variables. De forma general, todas las correlaciones son positivas y, además, como son mayores a 0.7, podemos decir que estas correlaciones son relativamente grandes. En el caso de correlaciones, puede volverse un poco difícil interpretar tantos números, sobre todo cuándo tenemos incluso más variables. Por lo anterior, hay diversas formas que se han creado para poder visualizar una matriz de correlaciones y poder obtener inferencias de formas más sencillas. En R hay distintos paquetes que pueden hacer este tipo de gráficos, pero para este laboratorio vamos a utilizar el paquete `corrplot`, por lo que vamos a proceder a cargarlo:

In [ ]:
library(corrplot)

La principal función del paquete `corrplot` se llama, al igual que el paquete `corrplot()`, donde podemos especificar el argumento `method` para poder visualizar la matriz de correlaciones que ya hemos calculado de diversas formas. Los siguientes gráficos presentan cuatro formas distintas de poder visualizar las correlaciones existentes entre nuestras variables:

In [ ]:
corrplot(mat.cor, method = "circle")

In [ ]:
corrplot(mat.cor, method = "pie")

In [ ]:
corrplot(mat.cor, method = "color")

In [ ]:
corrplot(mat.cor, method = "number")

Como podemos ver, aunque la forma de visualizar las correlaciones cambian de un gráfico a otro, la idea de fondo detrás del gráfico y las conclusiones a las cuales llegamos, son las mismas. Por todo esto, la forma en la cual se presentan las correlaciones son, por lo general, con base en los gustos de las personas y el público al cual se quiere llegar.

Hasta ahora solamente hemos estudiado la correlación de forma descriptiva, es decir, hemos descrito la correlacion presente en los datos que recolectamos pero, ¿qué pasa si queremos generalizar, o inferir, para la correlación en la población? En este caso, necesitamos hacer uso de los intervalos de confianza y las pruebas de hipótesis, como ya lo hemos visto, para poder realizar estas inferencias.

---

## Inferencias sobre $\rho$

Recordemos que los parámetros son letras griegas por lo que la correlación en la población se describe mediante la letra griega rho ($\rho$). Si tenemos un tamaño de muestra lo suficientemente grande, y si no hay valores extremos o una desviación muy grande con respecto a la normalidad condicional, podemos utilizar una aproximación normal para realizar una prueba de hipótesis sobre $\rho$ y calcular su intervalo de confianza correspondiente. Recordemos que las hipótesis serían, en este caso:

$$
H_0: \rho = 0 \qquad H_1: \rho \neq 0
$$

En otras palabras, la hipótesis nula nos dice que no hay una correlación lineal entre dos variables, mientras que, si rechazamos la hipótesis nula, podemos afirmar que existe una correlación lineal entre ambas variables, aunque no podemos determinar solamente con la prueba de hipótesis sobre la magnitud o la dirección de esta relación. Además, recordemos que correlación 0 no implica que las dos variables no estén relaciones de ninguna forma, solamente que la relación no es lineal. Por lo tanto, si dos variables se relacionan de forma cuadrática o curvilínea, entonces su correlación será cercana a 0 aunque si estén relacionadas. En el caso de los cálculos en R, podemos hacer uso de la función `cor.test()`, escribiendo las dos variables de interés, para realizar la prueba de hipótesis, obteniendo el p-value e intervalo de confianza respectivos. Procedamos a realizar la prueba de hipótesis para la correlación entre la esperanza de vida y las otras dos variables del conjunto de datos:

In [ ]:
cor.test(datos$life.exp, datos$sqrtTV.person)

In [ ]:
cor.test(datos$life.exp, datos$sqrtDoctors.person)

Como podemos notar, el comando `cor.test()` es muy similar a `t.test()`, donde nos devuelve un p-value y un intervalo de confianza con la confianza especifica, por default 95%. Analizando solamente los p-values, dado que ambos p-values son mucho menores al 5% (1.217 por 10 a la -11 y 1.048 por 10 a la -9, respectivamente) entonces podemos rechazar la hipótesis nula que la correlación es 0. En otras palabras, dado que rechazamos la hipótesis nula, podemos concluir que existe una correlación lineal entre la esperanza de vida al nacer y tanto la raíz cuadrada de la cantidad de televisores per cápita como la raíz cuadrada de la cantida de doctores per cápita. Ahora, ¿que forma tiene esta relación que sabemos que existe entre estas variables? Para poder contestar esta pregunta necesitamos analizar el intervalo de confianza. En ambos casos, podemos ver que el intervalo de confianza no contiene al 0, lo cual nos indica lo mismo que el valor de p, pero además sabemos que los intervalos de confianza son positivos, al ser mayores que 0, y además tenemos una idea sobre alrededor de que valores se encuentra esta correlación. Por lo tanto, para el caso de la correlación de ambas correlaciones, estas son positivas y tienen una magnitud entre alta y muy alta. En conclusión, la correlación es solamente otra técnica que utilizamos para describir la relación existente entre un par de variables, por lo que podemos usar gráficos, mediciones y métodos de inferencia para poder estudiar y analizar estas relaciones de forma más profunda.

---

## Ejercicio (4 pts.)

Con base en todo lo discutido anteriormente, si probamos que existe una correlación lineal entre la raíz cuadrada de la cantidad de televisores per cápita y la esperanza de vida al nacer, ¿por qué el gobierno de Costa Rica no subvenciona los televisores para aumentar la esperanza de vida? Justifique su respuesta y menciona posibles variables que no estamos tomando en cuenta que podrían explicar esta relación existente.

ESCRIBA SU RESPUESTA AQUI

---

## <span style="color: blue">Entrega del laboratorio</span> <a class="tocSkip"> 

Para hacer entrega de este laboratorio, debe descargarlo presionando en **File**, luego **Download as** y, por último, **Notebook (.ipynb)**, en el menú superior, debajo del logo de Jupyter. También puede descargarlo directamente desde el botón de **Download** en la barra con íconos para acceso rápido.

Luego de haber localizado el archivo descargado, debe cambiar el nombre del archivo descargado por su número de carné, de forma que, si su carné es **B12345**, su archivo debería llamarse `B12345.ipynb`. Una vez le haya cambiado el nombre al archivo, lo puede subir a Mediación Virtual, en el apartado correspondiente a este laboratorio.

<a href="https://creativecommons.org/licenses/by-sa/4.0/">
    <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-sa.png" width="88" height="31" />
</a>